<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/ML/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://a.teleboss.ru/play/c851b57a-ccd5-408c-9484-e3d00c79b46e

https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system/data?select=movies.csv

# Загрузка датасета с kaggle

In [1]:
!curl -L -o movie-recommendation-system.zip https://www.kaggle.com/api/v1/datasets/download/parasharmanas/movie-recommendation-system
!unzip -o /content/movie-recommendation-system.zip
!rm *.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  164M  100  164M    0     0   107M      0  0:00:01  0:00:01 --:--:--  142M
Archive:  /content/movie-recommendation-system.zip
  inflating: movies.csv              
  inflating: ratings.csv             


In [2]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100

In [3]:
ratings_full_df = pd.read_csv("/content/ratings.csv", nrows=200_000)
movies_df = pd.read_csv("/content/movies.csv")

In [4]:
ratings_full_df.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [5]:
movies_df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


# Подготовка pipline для обработки

In [26]:
X_ = ratings_full_df.drop("rating", axis=1)
y_ = ratings_full_df['rating']

In [27]:
onehot_columns = ['time'] # список колонок, которые будем кодировать c OneHotEncoder

In [28]:
col_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), onehot_columns),
    ],
    remainder='passthrough',          # Оставляем необрабатываемые колонки как есть, не удаляем их
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas


In [82]:
class NaivePredictor(ClassifierMixin, BaseEstimator):

    def __init__(self, a=0.5):
        self.rating_movie_mean_ = None  # Для хранения средней оценки фильма
        self.rating_user_mean_ = None  # Для хранения средней оценки которую ставит пользователь
        self.a = a  # Параметр, который будем менять при подборе по сетке GridSearchCV
        self.classes_ = [0] # Технический момент, никак не используется, но атрибут должен существовать для корректной работы GridSearchCV

    def fit(self, X, y):
        concatenated_df = pd.concat([X, y], axis=1)
        self.rating_movie_mean_ = concatenated_df.groupby('movieId')['rating'].mean()
        self.rating_user_mean_ = concatenated_df.groupby('userId')['rating'].mean()
        self.global_mean = y.mean()

        # Debugging: Print stats
        print("Global mean:", self.global_mean)
        print("Movie mean isna:", self.rating_movie_mean_.isna().sum())
        print("User mean isna:", self.rating_user_mean_.isna().sum())

        return self

    def predict(self, X):
        """
        Предсказывает средний рейтинг фильма для каждого примера.
        Если `movieId` нет в обучающей выборке, возвращает средний рейтинг по всем фильмам.
        """
        if self.rating_movie_mean_ is None:
            raise ValueError("Model is not fitted yet. Call `fit` before `predict`.")

        # Заполняем пропущенные значения средним рейтингом всех фильмов
        predictions = (X['movieId'].map(self.rating_movie_mean_) * self.a + X['userId'].map(self.rating_user_mean_) * (1 - self.a))

        print("Predictions before fillna:", predictions.isna().sum())
        predictions = predictions.fillna(self.global_mean)
        print("Predictions after fillna:", predictions.isna().sum())

        return predictions

In [69]:
pipe = Pipeline(
    [

        ("NaivePredictor", NaivePredictor())
    ]
)

In [71]:
splitter = KFold(
    n_splits=2,
    shuffle=True,
    random_state=42
)

In [72]:
param_grid = {
    "NaivePredictor__a": np.arange(start=0.1, stop=1.1, step=0.1)
}

In [77]:
search = GridSearchCV(pipe, param_grid,
                          cv=splitter, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10, return_train_score=True, error_score="raise")

search.fit(X_, y_)
print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Global mean: 3.56641
Movie mean isna: 0
User mean isna: 0
Best parameter (CV score=-0.71258):
{'NaivePredictor__a': 0.4}


In [70]:
def evaluate_model(y_true, y_predicted):
  result = pd.DataFrame.from_dict({
          "MAE": mean_absolute_error(y_true, y_predicted),
          "MSE": mean_squared_error(y_true, y_predicted),
          "MAPE": mean_absolute_percentage_error(y_true, y_predicted),
          "R2_score": r2_score(y_true, y_predicted )
  },
  orient='index', columns=['value'])
  return result

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    X_,
    y_,
    test_size=0.2, # <--- Доля теста к тотал данным
    random_state=42  # <--- Мешать ли данные или делить по индексам
)

In [81]:
evaluate_model(y_test, search.predict(X_test))

Predictions before fillna: 0
Predictions after fillna: 0


,value
MAE,0.681443
MSE,0.762313
MAPE,0.302812
R2_score,0.299740


In [7]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        movies_data,

    ):

        self.mlb = MultiLabelBinarizer()
        self.movies_data = movies_data

    def fit(self, X, y):

        X_train_copy = X.copy()
        X_train_copy = pd.concat([X_train_copy, y], axis = 1)
        X_train_copy = X_train_copy.merge(movies_df[['movieId', 'genres']], on='movieId', how='left')

        self.rating_user_mean = X_train_copy.groupby("userId", as_index=False)\
                                               .rating.mean()\
                                               .rename({"rating": "rating_user_mean"}, axis=1)

        self.rating_movie_mean = X_train_copy.groupby("movieId", as_index=False)\
                                                .rating.mean()\
                                                .rename({"rating": "rating_movie_mean"}, axis=1)


        self.mlb.fit(self.movies_data.genres.apply(lambda x: x.split('|')))


        mlb_encoded_genres = pd.DataFrame(self.mlb.transform(X_train_copy.genres.apply(lambda x: x.split('|'))), \
                                          columns=self.mlb.classes_)

        temp = pd.concat([X_train_copy.userId, mlb_encoded_genres.mul(y, axis=0)], axis=1).groupby("userId").agg(lambda x: np.mean([_ for _ in x if _ > 0]))
        self.user_genres_rating = temp.T.fillna(temp.mean(axis=1)).T
        self.mean_rating = y.mean()
        return self

    def transform(self, X:pd.DataFrame):

        X_copy= X.copy()
        X_copy = X_copy.merge(self.rating_user_mean, how='left', on='userId') \
                       .merge(self.rating_movie_mean, how='left', on='movieId') \
                       .merge(movies_df[['movieId', 'genres']], on='movieId', how='left').fillna(2.5)

        datetime_ = pd.to_datetime(X_copy.timestamp, unit='s')

        X_copy['year'] = datetime_.dt.year
        X_copy['month'] = datetime_.dt.month
        X_copy['day_of_week'] = datetime_.dt.day_of_week
        X_copy['time'] = pd.cut(datetime_.dt.hour,
                                include_lowest=True,
                                bins=[0, 5, 12, 17, 21, 24],
                                right=False,
                                ordered=False,
                                labels=['night', 'morining', 'afternoon', 'evening', 'night'])

        X_copy['genres'] = X_copy.genres.apply(lambda x: x.split('|'))

        mlb_encoded_genres = pd.DataFrame(self.mlb.transform(X_copy.genres), \
                                         columns=self.mlb.classes_)

        #X_copy = pd.concat([X_copy, mlb_encoded_genres], axis=1)
        X_copy = pd.concat([X_copy, self.user_genres_rating.merge(X_copy.userId, how='right', right_on='userId', left_index=True).iloc[:,:-1].apply(lambda col: col.fillna(col.mean()), axis=0) * mlb_encoded_genres], axis=1)

        """
        X_copy = X_copy.merge(pd.concat([mlb_encoded_genres,\
                                X_copy.userId], axis = 1)
                               .groupby("userId", as_index = False).mean(), \
                     how='left',
                     on='userId', \
                     suffixes = ('', '_gen_mean'))
        """


        X_copy.drop(columns=['timestamp', 'genres'], inplace=True)
        return X_copy.set_index(X.index)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

    def inverse_transform(self):
        pass

In [ ]:
final_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("linear_regression", Ridge())
    ]
)


In [ ]:
lasso_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer2(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Lasso())
    ]
)

ridge_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer2(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Ridge())
    ]
)

In [ ]:
param_grid = {
    "simple_model__alpha": np.linspace(start=0.1, stop=1000, num=100),
    "simple_model__max_iter": [100, 1000]
}

In [ ]:
%%time
for model in [lasso_pipe, ridge_pipe]:
    ### Передадим в GridSearchCV
    search = GridSearchCV(model, param_grid,
                          cv=splitter, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10)

    search.fit(X_, y_)

    print(f"Best parameter (CV score={search.best_score_:.5f}):")
    print(search.best_params_)

Fitting 2 folds for each of 200 candidates, totalling 400 fits


In [ ]:
evaluate_model(y_test, final_pipe.fit(X_train, y_train).predict(X_test))

In [ ]:
from sklearn.model_selection import KFold

splitter = KFold(
    n_splits=2,
    shuffle=True,
    random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate

cv_result = cross_validate(pipe,
                           X_, y_,
                           scoring='neg_mean_absolute_error',
                           cv=splitter, return_train_score=True)

cv_result

In [ ]:
param_grid = {}
search = GridSearchCV(final_pipe, param_grid,
                      cv=splitter, scoring='neg_mean_absolute_error')

In [ ]:
search.fit(X_, y_)
print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

In [ ]:
search.best_score_

In [ ]:
list(splitter.split(X_, y_))

In [ ]:
movies_df.genres.isna().sum()

In [ ]:
CustomTransformer2(movies_data=movies_df).fit(X_train, y_train).transform(X_test)

In [ ]:
 CustomTransformer2(movies_data=movies_df).fit(X_train, y_train).transform(X_test).isna().sum().loc[lambda x: x > 0]